In [0]:
import os
import geopandas as gpd
import pyspark.databricks.sql.functions as DBF
import pyspark.sql.functions as F

### What ST_ functions are available?

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {schema}")

In [0]:
%sql
USE CATALOG IDENTIFIER(:catalog);
USE SCHEMA IDENTIFIER(:schema);

In [0]:
%sql
SHOW FUNCTIONS LIKE 'ST_*'

### What is the definition of a given ST_ function?

In [0]:
%sql
DESCRIBE FUNCTION EXTENDED st_buffer

### About the Datasets
1. Local Government Areas - 2025 - Shapefile
- Digital boundaries are available in both the Geocentric Datum of Australia 2020 (GDA2020) and the Geocentric Datum of Australia 1994 (GDA94). GDA2020 was adopted as the new official national datum in 2017 and will be adopted gradually by organisations across Australia.

2. Statistical Area Level 3
- Clusters of SA2s representing regions with similar social/economic characteristics.
- Typical Population: 30,000–130,000 people.
- Analyzing regional patterns, such as employment or health.


In [0]:
# update to your preferred location
lga_data_path = "/Volumes/pamela_lim/dev/spatial_sql_lga"
sa3_data_path = "/Volumes/pamela_lim/dev/spatial_sql_mesh_block_boundaries/"

- Convert the GeoDataFrame to a Spark DataFrame
- If your geometry column is complex (e.g. Shapely objects), you will need to convert it to WKT or WKB format first since Spark does not natively understand geometry objects.

In [0]:
import geopandas as gpd
import pandas as pd

gdf_lga = gpd.read_file(f"{lga_data_path}/LGA_2025_AUST_GDA2020.shp")
gdf_sa3 = gpd.read_file(f"{sa3_data_path}/SA3_2021_AUST_GDA2020.shp")


# Convert geometry to WKT
gdf_lga['geometry'] = gdf_lga['geometry'].to_wkt()
gdf_sa3['geometry'] = gdf_sa3['geometry'].to_wkt()


# Convert to Spark DataFrame
lga_pdf = pd.DataFrame(gdf_lga)
lga_sdf = spark.createDataFrame(lga_pdf)

sa3_pdf = pd.DataFrame(gdf_sa3)
sa3_sdf = spark.createDataFrame(sa3_pdf)

# Write to Delta table
lga_sdf.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.lgas")
sa3_sdf.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.sa3")

In [0]:
%sql

-- 1. Explore LGA data
SELECT 
    lga_code25 as lga_code,
    lga_name25 as lga_name,
    ste_code21 as state_code,
    ste_name21 as state_name,
    aus_code21 as aus_code,
    aus_name21 as aus_name,
    areasqkm as area_sqkm,
    ST_NPoints(ST_GeomFromText(geometry)) as num_vertices
FROM lgas
LIMIT 10;

-- 2. ST_Transform: Area Calculation in Different Coordinate Reference System (CRS)
SELECT 
    lga_code25 as lga_code,
    lga_name25 as lga_name,
    ste_code21 as state_code,
    ste_name21 as state_name,
    areasqkm AS original_area,
    ST_Area(ST_Transform(ST_GeomFromText(geometry, 7844), 3112)) AS area_sqkm_gda2020_zone55,
    ST_Area(ST_Transform(ST_GeomFromText(geometry, 7844), 3857)) AS area_sqkm_web_mercator
FROM lgas
WHERE ste_name21 = 'Victoria'
ORDER BY areasqkm DESC
LIMIT 10;

-- 3. ST_Contains: Find all LGAs that contain a specific point (e.g., Sydney CBD coordinates)
SELECT 
    lga_code25 as lga_code,
    lga_name25 as lga_name,
    ste_name21 as state_name,
    areasqkm
FROM lgas
WHERE ST_Contains(
    ST_GeomFromText(geometry), 
    ST_Point(151.2093, -33.8688) -- Sydney CBD coordinates
);

-- 4. Find which LGA contains each SA3 (point-in-polygon)
SELECT
    s.SA3_CODE21,
    s.SA3_NAME21,
    s.STE_NAME21 as sa3_state,
    s.AREASQKM21 as sa3_area_sqkm,
    l.lga_code25,
    l.lga_name25,
    l.ste_name21 as lga_state,
    l.areasqkm as lga_area_sqkm
FROM sa3 s
JOIN lgas l ON ST_Contains(
    ST_GeomFromText(l.geometry),
    ST_Centroid(ST_GeomFromText(s.geometry))
)
LIMIT 20;


In [0]:
%sql

-- ============================================================================
-- QUICK START: H3 Indexing Examples
-- ============================================================================

-- H3 POINT-IN-POLYGON: Create H3 indexes for efficient spatial lookups
CREATE OR REPLACE TEMPORARY VIEW sa3_h3 AS
SELECT
    *,
    h3_longlatash3(
        ST_X(ST_Centroid(ST_GeomFromText(geometry))),
        ST_Y(ST_Centroid(ST_GeomFromText(geometry))),
        9
    ) as h3_centroid_9
FROM sa3;

CREATE OR REPLACE TEMPORARY VIEW lgas_h3 AS
SELECT
    *,
    h3_longlatash3(
        ST_X(ST_Centroid(ST_GeomFromText(geometry))),
        ST_Y(ST_Centroid(ST_GeomFromText(geometry))),
        9
    ) as h3_centroid_9
FROM lgas;

-- 2. H3-BASED SPATIAL JOIN: Use H3 for pre-filtering before spatial operations
SELECT
    s.SA3_CODE21,
    s.SA3_NAME21,
    l.lga_code25,
    l.lga_name25,
    s.h3_centroid_9
FROM sa3_h3 s
JOIN lgas_h3 l ON s.h3_centroid_9 = l.h3_centroid_9  -- Fast H3 comparison
WHERE ST_Contains(
    ST_GeomFromText(l.geometry),
    ST_Centroid(ST_GeomFromText(s.geometry))
)
LIMIT 10;

-- 3. H3 TESSELLATION: Cover SA3 areas with H3 cells
-- Generate H3 cells that cover each SA3 polygon
SELECT
    SA3_CODE21,
    SA3_NAME21,
    h3_cell,
    h3_resolution(h3_cell) as h3_resolution,
    h3_boundaryaswkt(h3_cell) as h3_boundary
FROM sa3
LATERAL VIEW explode(
    h3_coverash3(
        geometry,
        9  -- H3 resolution 9 (~173km²)
    )
) AS h3_cell
WHERE STE_NAME21 = 'New South Wales'
LIMIT 100;

-- 4. Use H3 distance functions for spatial proximity analysis
SELECT
    s.SA3_CODE21,
    s.SA3_NAME21,
    l.lga_code25,
    l.lga_name25,
    h3_try_distance(s.h3_centroid_9, l.h3_centroid_9) as h3_grid_distance
FROM sa3_h3 s
JOIN lgas_h3 l ON h3_try_distance(s.h3_centroid_9, l.h3_centroid_9) <= 3
WHERE s.STE_NAME21 = 'New South Wales'
ORDER BY h3_grid_distance
LIMIT 15;